In [2]:
import numpy as np
from itertools import product
import copy
import sys

In [2]:
#Stolen Code
class Vehicle():
    def __init__(self, id):
        self.id = id
        self.history = []
        self.batch_num = 0
        self.last_avail_time = 0

    def bind_fn(self, fx): #Fx is expected to be of type f()(current_loc, current_time, start_loc, start_time, end_loc). Assumes the data set does not contain self impossible rides, ie, that if you start a ride on time you can finish it as well
        self.fn = fx

    def getEndTime(self, start_loc, start_time, end_loc):
        return abs(end_loc[0] - start_loc[0]) + abs(end_loc[1] - start_loc[1]) + start_time - 1

    def bind(self, batch, ride): #Probably don't need all this info but it is nice to keep track of. End time here will be the ACTUAL finish time of the vehicle, as will start time, capped to a min of ride_start_time. Start time is the delta step when the vehicle is allowed to make its first move
        self.ride_id = ride[-1]
        self.history.append(self.ride_id)
        self.start_loc = (ride[0], ride[1])
        self.end_loc = (ride[2], ride[3])
        self.start_time = (ride[4] if self.last_avail_time < ride[4] else self.last_avail_time)
        self.end_time = self.getEndTime(self.start_loc, self.start_time, self.end_loc)
        self.last_avail_time = self.end_time + 1
        self.batch = batch
        self.batch_num += 1 #Used to externally slice the data set to pass proper batch reference to bind()


    def score(self):
        try:
            tmp = [self.fn(self.end_loc, self.end_time, (e[0],e[1]), e[4], (e[2],e[3])) for e in self.batch] #Batch is assumed to be a list of tuples of type (ride_id, start_loc, start_time, end_loc, etc...)
            idx = max(range(len(tmp)), key=tmp.__getitem__)
            ret = copy.deepcopy(self.batch[idx])
            del(self.batch[idx])
            return ret
        except AttributeError:
            print("Missing a bind")
            return None
        except ValueError as e: #max(empty), batch is depleted. This can only happen in the very last batch when the total number of rides isn't evenly divded by the number of vehicles.
            return None

In [3]:
#Some example functions to calculate score
import math

def getDist(current_loc, start_loc):
    return abs(start_loc[0] - current_loc[0]) + abs(start_loc[1] - current_loc[1])

def fn_bonus_greed_only(current_loc, current_time, start_loc, start_time, end_loc):
    #Check to see if the ride is actually possible to start on time
    if getDist(current_loc, start_loc) + current_time  > start_time:
        return 0
    return getDist(start_loc, end_loc) #No need to add bonus to all values

def fn_ratio_greed(current_loc, current_time, start_loc, start_time, end_loc):
    return (getDist(start_loc, end_loc) + (2 if getDist(current_loc, start_loc) + current_time > start_time else 0 )/getDist(current_loc, start_loc))

def fn_log_greed(current_loc, current_time, start_loc, start_time, end_loc):
    return math.log(getDist(start_loc, end_loc) + (2 if getDist(current_loc, start_loc) + current_time > start_time else 0 ),getDist(current_loc, start_loc))


In [4]:
with open('data/b_should_be_easy.in') as file:
    data = file.read().split()

num_rows = int(data[0])
num_cols = int(data[1])
num_vehicles = int(data[2])
num_rides = int(data[3])
bonus = int(data[4])
total_steps = int(data[5])
num_batches = int(num_rides/num_vehicles)

rides = []

for i in range(int(num_rides)):
    rides.append([int(x) for x in data[6+i*6:12+i*6]]+[i])

# for i, r in enumerate(rides):
#     start_coords = tuple([r[0],r[1]])
#     end_coords = tuple([r[2],r[3]])
#     earliest_start = r[4]
#     latest_end = r[5]
#     rides[i] = [start_coords, end_coords, earliest_start, latest_end]

# rides = np.array(rides)


In [5]:
rides[0]

[395, 43, 296, 955, 244, 3620, 0]

In [6]:
def sim_score(c,ride):
    return c[0]*(ride[0]+ride[1])+c[1]*(ride[2]+ride[3])+c[2]*ride[4]+c[3]*ride[5]

In [15]:
c = [1,1,1,1]
sorted_rides = []

# [start_x, start_y, end_x, end_y, start_time, end_time, index]
sorted_rides = sorted(rides,key = lambda x : sim_score(c,x))

In [9]:
batches = [sorted_rides[num_vehicles*i:num_vehicles*(i+1)] for i in range(num_batches)]
batches.append(sorted_rides[num_vehicles*num_batches:])
history = []
vehicles = [Vehicle(i) for i in range(num_vehicles)]
for i, v in enumerate(vehicles):
    v.bind(batches[1],batches[0][i])
#     print(batches[1])
    v.bind_fn(fn_ratio_greed)
while(vehicles):
    tmp = [v.end_time for v in vehicles]
    idx = min(range(len(tmp)), key=tmp.__getitem__)
    cur_vehicle = vehicles[idx]
    next_ride = cur_vehicle.score()
#     print(cur_vehicle)
    if(next_ride != None):
        cur_vehicle.bind(batches[cur_vehicle.batch_num+1],next_ride)
    else:
        history.append(cur_vehicle.history)
        vehicles.remove(cur_vehicle)
print(history)

[[290, 139, 199], [224, 222, 16], [167, 282, 98], [246, 241, 205], [187, 235, 255], [60, 109, 20], [286, 197, 238], [38, 29, 181], [260, 32, 112], [85, 284, 219], [285, 147, 31], [296, 258, 250], [234, 271, 266], [79, 118, 51], [261, 90, 264], [267, 254, 223], [213, 291, 115], [168, 243, 78], [4, 295, 292], [164, 214, 196], [63, 230, 265], [136, 283, 55], [91, 166, 242], [65, 114, 182], [105, 28, 106], [200, 26, 289], [88, 50, 155], [56, 188, 12], [249, 73, 192], [44, 117, 145], [263, 128, 152], [24, 14, 67], [95, 81, 280], [99, 244, 104], [74, 248, 66], [281, 53, 59], [75, 39, 141], [13, 298, 176], [245, 268, 142], [57, 45, 17], [216, 111, 71], [103, 83, 100], [58, 158, 228], [276, 102, 43], [125, 3, 252], [137, 61, 215], [193, 239, 154], [19, 253, 297], [107, 23, 30], [133, 237, 120], [203, 174, 288], [299, 190, 251], [34, 201, 231], [15, 40, 126], [212, 204, 206], [41, 170, 262], [184, 89, 47], [150, 269, 279], [18, 76, 186], [33, 96, 256], [54, 131, 229], [294, 35, 270], [129, 191,

In [12]:
print(rides[290],rides[139],rides[199])

[285, 20, 605, 565, 5193, 8091, 290] [148, 374, 546, 448, 16900, 17773, 139] [681, 206, 756, 220, 17523, 17742, 199]


In [14]:
print(rides[221],rides[68],rides[123])

[518, 656, 201, 494, 186, 712, 221] [188, 405, 689, 968, 11576, 13755, 68] [296, 47, 137, 570, 22183, 23150, 123]


In [10]:
from JCPEOptimization import algorithm

FileNotFoundError: [Errno 2] No such file or directory: '-f'